# Perfoming clustering of customers based on RFM (Recency, frequency, Monetary) analysis.

# Clustering was done on categorical variable 'sex' in the  dataset Retail1.csv.

## Loading required libraries

In [95]:
import pandas as pd
import numpy as np
import sklearn
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

## Loading dataset

In [96]:
# Load the dataset
df = pd.read_csv('Retail1.csv')
print(df.head())

   Unnamed: 0 InvoiceNo StockCode                          Description  \
0           0    570715     21014        SWISS CHALET TREE DECORATION    
1           1    573167     21014        SWISS CHALET TREE DECORATION    
2           2    570715     35967   FOLK ART METAL STAR T-LIGHT HOLDER   
3           3    570715     21824  PAINTED METAL STAR WITH HOLLY BELLS   
4           4    570715     22065       CHRISTMAS PUDDING TRINKET POT    

   Quantity     InvoiceDate  UnitPrice  CustomerID         Country   sex  
0        24  10/12/11 10:23       0.29      118287  United Kingdom  male  
1        24   10/28/11 9:29       0.29      118287  United Kingdom  male  
2        36  10/12/11 10:23       0.38      118287  United Kingdom  male  
3        24  10/12/11 10:23       0.39      118287  United Kingdom  male  
4        48  10/12/11 10:23       0.39      118287  United Kingdom  male  


In [97]:
#checking quantity and priceless than zero
print((df['Quantity'] <= 0).any())
print((df['UnitPrice'] <= 0).any())


True
True


In [98]:
#change into category
df['sex'] = df['sex'].astype('category')
#summary statistics of column Quantity and Unit Price
print(df[["Quantity", "UnitPrice"]].describe())
#Filtering Quantity and Unit Price wgreater than O.
df = df[(df['Quantity'] > 0) & (df['UnitPrice'] > 0)]

            Quantity      UnitPrice
count  406829.000000  406829.000000
mean       12.061303       3.460471
std       248.693370      69.315162
min    -80995.000000       0.000000
25%         2.000000       1.250000
50%         5.000000       1.950000
75%        12.000000       3.750000
max     80995.000000   38970.000000


In [99]:
# separate date, month, year, daysOfWeek column from InvoiceDate
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df['Date'] = df['InvoiceDate'].dt.date
df['Month'] = df['InvoiceDate'].dt.month
df['Year'] = df['InvoiceDate'].dt.year
df['DayOfWeek'] = df['InvoiceDate'].dt.day_name()

In [100]:
# Convert the columns to the 'category' data type
df['Month'] = df['Month'].astype('category')
df['Year'] = df['Year'].astype('category')
df['DayOfWeek'] = df['DayOfWeek'].astype('category')

In [101]:
# Create the 'Total Sales' column
df['Total Sales'] = df['Quantity'] * df['UnitPrice']
df = df[(df['Total Sales'] > 0)]
CountrySales = pd.DataFrame(df.groupby('Country')['Total Sales'].sum()).reset_index()

In [102]:
# Total Sales By Month
sales = df.groupby(["Year","Month"])["Total Sales"].sum().reset_index()
sales  = sales[(sales['Total Sales'] > 0)]
import calendar
sales['Month'] = sales['Month'].apply(lambda x: calendar.month_abbr[x])
sales['Month'] = sales['Month'].astype(str)+ '' + sales ['Year'].astype(str)
sales = sales.drop("Year", axis=1)

In [103]:
#RFA Analysis
#Total Sales(Monetary)
df['CustomerID'] = df['CustomerID'].astype(str)
rfm_ds_m = df.groupby('CustomerID')['Total Sales'].sum()
rfm_ds_m.reset_index()
rfm_ds_m.columns = ['CustomerID', 'Total Amount']

In [104]:
#frequency of purchase
df['CustomerID'] = df['CustomerID'].astype(str)
rfm_ds_f = df.groupby('CustomerID')['InvoiceNo'].count()
rfm_ds_f = rfm_ds_f.reset_index()
rfm_ds_f.columns = ['CustomerID', 'Frequency']

In [105]:
#Recent Purchase
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'],format='%d-%m-%Y %H:%M')
max_date = max(df['InvoiceDate'])
df['Diff'] = max_date - df['InvoiceDate']
rfm_ds_p = df.groupby('CustomerID')['Diff'].min()
rfm_ds_p = rfm_ds_p.reset_index()
rfm_ds_p.columns = ['CustomerID','Diff']
rfm_ds_p['Diff'] = rfm_ds_p['Diff'].dt.days

In [106]:
# Group by 'CustomerID' and aggregate 'Sex' as a list
grouped_data = df.groupby('CustomerID')['sex'].unique().reset_index()
new_sex = pd.DataFrame({'CustomerID': grouped_data['CustomerID'], 'sex': grouped_data['sex'].str[0]})

In [107]:
rfm_ds_final = pd.merge(rfm_ds_m,rfm_ds_f,on='CustomerID',how='inner')
rfm_ds_final = pd.merge(rfm_ds_final,rfm_ds_p,on='CustomerID',how='inner')
rfm_ds_final = pd.merge(rfm_ds_final,new_sex,on='CustomerID',how='inner')
rfm_ds_final.columns = ['CustomerID','Amount', 'Frequency', 'Recency', 'sex']
Cus_ID = rfm_ds_final['CustomerID']

In [108]:
# Encode the 'sex' column
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
sex=rfm_ds_final['sex']
rfm_ds_final['sex_encoded'] = encoder.fit_transform(rfm_ds_final['sex'])
#Changing into categorical
rfm_ds_final['sex_encoded'] = rfm_ds_final['sex_encoded'].astype('category')
# Access the encoded 'sex' column
sex_encoded = rfm_ds_final['sex_encoded']
sex_encoded.head()

0    1
1    1
2    0
3    1
4    1
Name: sex_encoded, dtype: category
Categories (2, int64): [0, 1]

In [109]:
rfm_ds_final.describe()

,Amount,Frequency,Recency
count,4358.000000,4358.000000,4358.000000
mean,2044.838895,91.299679,91.589720
std,8965.929599,227.998127,100.027635
min,3.750000,1.000000,0.000000
25%,306.592500,17.000000,17.000000
50%,668.395000,41.000000,50.000000
75%,1653.210000,99.000000,141.750000
max,280206.020000,7847.000000,373.000000


Mean of all the three varaiables are greater than median. Threre are some outilers in the data, which has greater values.

# Handling outliers

In [110]:

# Handling Outliers
Q1 = rfm_ds_final.Amount.quantile(0.05)
Q3 = rfm_ds_final.Amount.quantile(0.95)
IQR = Q3 - Q1
rfm_ds_final = rfm_ds_final[(rfm_ds_final.Amount >= Q1 - 1.5*IQR) & (rfm_ds_final.Amount <= Q3 + 1.5*IQR)]

Q1 = rfm_ds_final.Recency.quantile(0.05)
Q3 = rfm_ds_final.Recency.quantile(0.95)
IQR = Q3 - Q1
rfm_ds_final = rfm_ds_final[(rfm_ds_final.Recency >= Q1 - 1.5*IQR) & (rfm_ds_final.Recency <= Q3 + 1.5*IQR)]

Q1 = rfm_ds_final.Frequency.quantile(0.05)
Q3 = rfm_ds_final.Frequency.quantile(0.95)
IQR = Q3 - Q1
rfm_ds_final = rfm_ds_final[(rfm_ds_final.Frequency >= Q1 - 1.5*IQR) & (rfm_ds_final.Frequency <= Q3 + 1.5*IQR)]
print(rfm_ds_final.head())

X = rfm_ds_final[['Amount', 'Frequency', 'Recency', 'sex_encoded']]

  CustomerID  Amount  Frequency  Recency     sex sex_encoded
0    1112354  132.48          8      231    male           1
1    1114911  838.10         33        2    male           1
2    1118287  822.84         26       58  female           0
4     112347  389.12         30        1    male           1
5     112349  525.68         30       18  female           0


In [111]:
print(X.head())


   Amount  Frequency  Recency sex_encoded
0  132.48          8      231           1
1  838.10         33        2           1
2  822.84         26       58           0
4  389.12         30        1           1
5  525.68         30       18           0


In [112]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4275 entries, 0 to 4357
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   Amount       4275 non-null   float64 
 1   Frequency    4275 non-null   int64   
 2   Recency      4275 non-null   int64   
 3   sex_encoded  4275 non-null   category
dtypes: category(1), float64(1), int64(2)
memory usage: 137.9 KB


## Performing heat map for checking correlation and multicollinearity.

In [113]:
#Heat Map for determing the correlation of variables.
import plotly.express as px

# Create a correlation matrix
correlation_matrix = X.corr()
# Generate heatmap using Plotly
fig = px.imshow(correlation_matrix,
                x=correlation_matrix.columns,
                y=correlation_matrix.columns)

# Display the heatmap
fig.show()

Heat Map is not showing sex as it is categorical variable in the above dataframe, we will sacle the variable which will change it in the float and we will perform heat map again later.

In [114]:

#MinMaxScaling
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
rfm_ds_scaled = scaler.fit_transform(X)
rfm_ds_scaled = pd.DataFrame(rfm_ds_scaled)
rfm_ds_scaled.columns = ['Amount', 'Frequency', 'Recency', 'sex']
print(rfm_ds_scaled.head())

     Amount  Frequency   Recency  sex
0  0.009175   0.009873  0.619303  1.0
1  0.059464   0.045134  0.005362  1.0
2  0.058376   0.035261  0.155496  0.0
3  0.027465   0.040903  0.002681  1.0
4  0.037198   0.040903  0.048257  0.0


In [115]:
rfm_ds_scaled.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4275 entries, 0 to 4274
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Amount     4275 non-null   float64
 1   Frequency  4275 non-null   float64
 2   Recency    4275 non-null   float64
 3   sex        4275 non-null   float64
dtypes: float64(4)
memory usage: 133.7 KB


In [116]:
import plotly.express as px
import numpy as np

# Create a correlation matrix
correlation_matrix = rfm_ds_scaled.corr()

# Generate heatmap using Plotly
fig = px.imshow(correlation_matrix,
                x=correlation_matrix.columns,
                y=correlation_matrix.columns,
                zmin=-1,  # Set minimum value of color scale
                zmax=1)  # Set maximum value of color scale

# Add custom annotations to show correlation values
fig.update_layout(annotations=[
    dict(
        x=i,
        y=j,
        text=np.around(correlation_matrix.values[i][j], decimals=2),
        showarrow=False,
        font=dict(size=9),
    )
    for i in range(len(correlation_matrix.columns))
    for j in range(len(correlation_matrix.columns))
])

# Display the heatmap
fig.show()


In [117]:
!pip install kmodes



## Determing silhoute score for each clusters

In [118]:
import numpy as np
import plotly.offline as pyo
import plotly.graph_objs as go
from kmodes.kprototypes import KPrototypes
from sklearn.metrics import silhouette_score
num_clusters = list(range(2, 9))
silhouette_avg = []
# calculate cost values for each number of clusters (2 to 8)
for k in num_clusters:
    try:
        kproto = KPrototypes(n_jobs=-1, n_clusters=k, init='Huang', random_state=42)
        kproto.fit_predict(rfm_ds_scaled, categorical=[3])
        cluster_labels = kproto.labels_
        silhouette_avg.append(silhouette_score(rfm_ds_scaled, cluster_labels))
        print('Cluster initiation: {}'.format(k))
    except:
        break

trace = go.Scatter(x=num_clusters, y=silhouette_avg, mode='lines+markers', name='Silhouette Score')
data = [trace]
layout = go.Layout(title='Silhouette Curve', xaxis=dict(title='Number of Clusters'), yaxis=dict(title='Score'))
fig = go.Figure(data=data, layout=layout)
pyo.plot(fig, filename='silhouette.html')



Cluster initiation: 2
Cluster initiation: 3
Cluster initiation: 4
Cluster initiation: 5
Cluster initiation: 6
Cluster initiation: 7
Cluster initiation: 8


'silhouette.html'

In [119]:
fig.show()

As max silhoute score is only 0.56 which indicates that clusters are not properly seperated from each otherThe value of silhoute score ranges from 0 to 1. Here in the above graph cluster 5 have maxium silhoute score. We will perform PCA later for the dimensinal reduction.

# Lets see what will the optimal number of cluster, elbow curve shows.

In [120]:
num_clusters = list(range(2, 9))
cost_values = []
for k in num_clusters:
    kproto = KPrototypes(n_jobs=-1, n_clusters=k, init='Huang', random_state=42)
    kproto.fit_predict(rfm_ds_scaled, categorical=[3])
    cost_values.append(kproto.cost_)
    cluster_labels = kproto.labels_
    print('Cluster initiation: {}'.format(k))
trace = go.Scatter(x=num_clusters, y=cost_values, mode='lines+markers', name='Elbow curve')
data = [trace]
layout = go.Layout(title='Elbow curve', xaxis=dict(title='Number of Clusters'), yaxis=dict(title='Cost'))
fig1 = go.Figure(data=data, layout=layout)
fig1.show()

Cluster initiation: 2
Cluster initiation: 3
Cluster initiation: 4
Cluster initiation: 5
Cluster initiation: 6
Cluster initiation: 7
Cluster initiation: 8


# Based on the silhoutte and elbow method, the optimum number of cluster is 5. We will cluster the dataset using Kprototype clustering algorithm which is effective for categorical variables as K-means clustering is only effective for numerical continuous variable.


In [121]:
# we set the number of clusters to 6
kproto = KPrototypes(n_jobs=-1, n_clusters=5 , init='Huang', n_init = 25, random_state=42)#n_init -number of centroid initialization
kproto.fit_predict(rfm_ds_scaled, categorical=[3])
# store cluster labels
cluster_labels = kproto.labels_
# add clusters to dataframe
#rfm_ds_scaled["cluster"] = cluster_labels

#Adding customer in the dataframe
#rfm_ds_scaled['customer'] = Cus_ID
#print(rfm_ds_scaled.head())


As metioned earlier, in our dataset the silhoute score in the clusters were not very high which indicate that our clusters are not seperated proeply so we will perform PCA(Pricipal component analysis to reduce the dimensions of the dataset). Lets see how it performs

In [122]:

#Cluster Exploration
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
pca_df = pca.fit_transform(rfm_ds_scaled)



In [123]:
explained_variance_ratio = pca.explained_variance_ratio_
print(explained_variance_ratio)

[0.69820537 0.21832914]


Principal component 1 expresses 70% and PC2 explained 22% variance of our dataset.

In [124]:
print(rfm_ds_scaled.head())

     Amount  Frequency   Recency  sex
0  0.009175   0.009873  0.619303  1.0
1  0.059464   0.045134  0.005362  1.0
2  0.058376   0.035261  0.155496  0.0
3  0.027465   0.040903  0.002681  1.0
4  0.037198   0.040903  0.048257  0.0


In [125]:
import numpy as np
np.set_printoptions(suppress=True)
print(pca.components_)

pca_dataframe = pd.DataFrame(pca_df)
print(pca_dataframe.head())

[[-0.00061619 -0.00185196  0.01233783 -0.99992198]
 [-0.21640514 -0.25228425  0.94306507  0.0122369 ]]
          0         1
0 -0.482888  0.397207
1 -0.490559 -0.201558
2  0.511234 -0.069482
3 -0.490565 -0.196094
4  0.509913 -0.167455


In [126]:
print(pca_dataframe.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4275 entries, 0 to 4274
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       4275 non-null   float64
 1   1       4275 non-null   float64
dtypes: float64(2)
memory usage: 66.9 KB
None


In [127]:
pca_df

array([[-0.48288833,  0.39720736],
       [-0.49055933, -0.20155759],
       [ 0.51123394, -0.06948215],
       ...,
       [ 0.51535159,  0.25717849],
       [ 0.50961188, -0.18350041],
       [-0.48917803, -0.09008985]])

In [128]:
#Plotting PCA
fig = px.scatter(x=pca_df[:, 0], y=pca_df[:, 1], color=cluster_labels)
fig.show()


In [129]:
# add clusters to dataframe
rfm_ds_scaled["cluster"] = cluster_labels

#Adding customer in the dataframe
rfm_ds_scaled['customer'] = Cus_ID
print(rfm_ds_scaled.head())

     Amount  Frequency   Recency  sex  cluster customer
0  0.009175   0.009873  0.619303  1.0        2  1112354
1  0.059464   0.045134  0.005362  1.0        4  1114911
2  0.058376   0.035261  0.155496  0.0        1  1118287
3  0.027465   0.040903  0.002681  1.0        4   112346
4  0.037198   0.040903  0.048257  0.0        1   112347


In [130]:
# size of each cluster
print(rfm_ds_scaled["cluster"].value_counts())
print(rfm_ds_scaled["sex"].value_counts())

1    1448
4    1435
2     530
0     522
3     340
Name: cluster, dtype: int64
1.0    2190
0.0    2085
Name: sex, dtype: int64


In [131]:
#Clusters for numeric variables
fig = px.scatter_matrix(rfm_ds_scaled,
    dimensions=['Amount', 'Frequency','Recency','customer','sex'],
    color="cluster",
    hover_data=['customer','sex'])
fig.show()

Cluster3 is has highest amount, frequency and low recency value as compared to others.

In [132]:
#Clusters for categorical  variable
fig = px.histogram(rfm_ds_scaled, x="sex", color="cluster", barmode="group")
fig.show()

In [133]:
#look at stastitics
rfm_ds_scaled[rfm_ds_scaled["cluster"]==0].describe()


,Amount,Frequency,Recency,sex,cluster
count,522.000000,522.000000,522.000000,522.0,522.0
mean,0.034403,0.035936,0.674463,0.0,0.0
std,0.046775,0.041939,0.180609,0.0,0.0
min,0.000641,0.000000,0.388740,0.0,0.0
25%,0.012144,0.011283,0.512064,0.0,0.0
50%,0.021845,0.022567,0.659517,0.0,0.0
75%,0.036597,0.041961,0.825737,0.0,0.0
max,0.557949,0.291961,1.000000,0.0,0.0


In [134]:
rfm_ds_scaled[rfm_ds_scaled["cluster"]==4].describe()

,Amount,Frequency,Recency,sex,cluster
count,1435.000000,1435.000000,1435.000000,1435.0,1435.0
mean,0.072746,0.081426,0.121871,1.0,4.0
std,0.069355,0.071218,0.098435,0.0,0.0
min,0.000802,0.000000,0.000000,1.0,4.0
25%,0.024588,0.026093,0.042895,1.0,4.0
50%,0.049472,0.059238,0.096515,1.0,4.0
75%,0.098101,0.119182,0.187668,1.0,4.0
max,0.525338,0.365303,0.420912,1.0,4.0


In [135]:
rfm_ds_scaled[rfm_ds_scaled["cluster"]==3].describe()

,Amount,Frequency,Recency,sex,cluster
count,340.000000,340.000000,340.000000,340.000000,340.0
mean,0.397350,0.461682,0.044898,0.661765,3.0
std,0.215249,0.196441,0.051785,0.473806,0.0
min,0.073823,0.011283,0.000000,0.000000,3.0
25%,0.239473,0.329337,0.008043,0.000000,3.0
50%,0.348951,0.435825,0.026810,1.000000,3.0
75%,0.507341,0.578632,0.061662,1.000000,3.0
max,1.000000,1.000000,0.402145,1.000000,3.0


In [136]:
#Cluster3 is the best cluster among all.